### Importing the dataset

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
wines = pd.read_csv("Dataset/last/XWines_Slim_1K_wines.csv", encoding="utf-8")
ratings = pd.read_csv("Dataset/last/XWines_Slim_150K_ratings.csv", low_memory=False)

In [3]:
wines.head(2)

In [4]:
ratings.head(5)

### Preprocessing

#### Joining the datasets

In [5]:
wines_ratings = pd.merge(wines, ratings, on="WineID")

#### Selecting the variables

In [6]:
wines_ratings.columns

In [7]:
wines_ratings.info()

In [8]:
wines_ratings = wines_ratings[
    [
        "RatingID",
        "UserID",
        "Rating",
        "WineName",
        "Vintage",
        "Type",
        "Elaborate",
        "Grapes",
        "Harmonize",
        "ABV",
        "Body",
        "Acidity",
        "Country",
        "RegionName",
        "WineryName",
    ]
]

#### Replacing special characters

In [9]:
wines_ratings.head(3)

In [10]:
wines_ratings['Grapes'].head()

In [11]:
wines_ratings['Grapes'] = wines_ratings['Grapes'].str.replace("[", " ").str.replace("]", " ").str.replace("'", "").str.strip()
wines_ratings['Grapes'].head()

In [12]:
wines_ratings['Body'] = wines_ratings['Body'].str.replace('-', '')
wines_ratings['Body'].head()

In [13]:
wines_ratings["Harmonize"] = wines_ratings["Harmonize"].str.replace("[", " ").str.replace("]", " ").str.replace("'", "").str.strip()
wines_ratings["Harmonize"].head()

In [14]:
wines_ratings.head(3)

In [15]:
def check_special_characters(df, column_name, special_characters_pattern):
    rows_with_special_characters = df[df[column_name].str.contains(special_characters_pattern, regex=True, na=False)]
    row_count = len(rows_with_special_characters)
    print("Number of rows with special characters in {}: {}".format(column_name, row_count))
    print(rows_with_special_characters[[column_name]].drop_duplicates().sort_values(by=column_name))

In [16]:
import re
def remove_special_characters(df, column_name, characters_to_remove):
    df[column_name] = df[column_name].str.replace(f'[{re.escape(characters_to_remove)}]', ' ', regex=True)
    return df

In [17]:
characters_to_remove = '[!"#$%°&\'()*+,-./:;<=>?@[\\]^_`{|}~]'

In [18]:
check_special_characters(wines_ratings, 'WineName', characters_to_remove)

In [19]:
wines_ratings = remove_special_characters(wines_ratings, 'WineName', characters_to_remove)
check_special_characters(wines_ratings, 'WineName', characters_to_remove)

In [20]:
check_special_characters(wines_ratings, 'RegionName', characters_to_remove)

In [21]:
wines_ratings = remove_special_characters(wines_ratings, 'RegionName', characters_to_remove)
check_special_characters(wines_ratings, 'RegionName', characters_to_remove)

In [22]:
check_special_characters(wines_ratings, 'WineryName', characters_to_remove)

In [23]:
wines_ratings = remove_special_characters(wines_ratings, 'WineryName', characters_to_remove)
check_special_characters(wines_ratings, 'WineryName', characters_to_remove)

#### Null values

In [24]:
wines_ratings.isnull().sum()

#### Splitting list variables

In [25]:
wines_ratings.head(3)

In [26]:
def split_and_expand_columns(df, column_name, delimiter="/"):
    expanded_columns = df[column_name].str.split(delimiter, expand=True)
    new_columns = [f"{column_name}{i+1}" for i in range(expanded_columns.shape[1])]
    expanded_columns.columns = new_columns

    df = pd.concat([df, expanded_columns], axis=1)
    df = df.fillna("")
    df = df.drop(columns=[column_name])

    return df

In [27]:
wines_ratings['Harmonize'].head()

In [28]:
wines_ratings = split_and_expand_columns(wines_ratings, "Harmonize", delimiter=", ")

In [29]:
wines_ratings.iloc[:, -6:].head()

In [30]:
wines_ratings = split_and_expand_columns(wines_ratings, "Grapes", delimiter=", ")

In [31]:
wines_ratings.sample(15).iloc[:, -9:]

In [32]:
wines_ratings['Elaborate'].value_counts()

In [33]:
wines_ratings = split_and_expand_columns(wines_ratings, "Elaborate")

In [34]:
wines_ratings.sample(10).iloc[:, -2:]

In [35]:
wines_ratings['Type'].value_counts()

In [36]:
wines_ratings = split_and_expand_columns(wines_ratings, "Type")

In [37]:
wines_ratings.sample(10).iloc[:, -2:]

#### Alcohol beverage outliers

In [38]:
wines_ratings['ABV'].describe()

In [39]:
wines_ratings['ABV'].value_counts()

In [40]:
def transform_values(df, column_name):
    df[column_name] = np.where(df[column_name] > 15, 15, df[column_name])
    df[column_name] = np.where(df[column_name] < 8, 8, df[column_name])
    return df

In [41]:
wines_ratings = transform_values(wines_ratings, 'ABV')

In [42]:
wines_ratings['ABV'].describe()

In [43]:
wines_ratings['ABV'].value_counts()

### Exporting

In [44]:
wines_ratings.to_csv('Dataset/XWines_with_Ratings.csv', index=False)